In [17]:
import pandas as pd
import datetime
import collections
import matplotlib.pylab as plt
import numpy as np
#make cleaned dataframes, with some preprocessing
#use the conversation allairlines file that niels shared somewhere
#then just run this function with the name of the file you want to open
def make_csv(filename):
    conv = pd.read_csv('Conversations allairlines/'+filename, encoding='utf-16')
    conv = conv.dropna(thresh=5)
    conv['user.id_str'] = conv['user.id_str'].astype(str)
    conv['created_at'] = pd.to_datetime(conv['created_at'])
    conv = conv.sort_values(['conversationnr', 'created_at'])
    conv = conv.reset_index()
    return conv

In [12]:
#klm and british air are not in the conversation allairlines file, so I open them seperately and do some more preprocessing
klm_conv = pd.read_csv('KLMconvers.csv', encoding='utf-16').dropna(thresh=5)
BA_conv = pd.read_csv('BAWconvers.csv', encoding='utf-16').dropna(thresh=5)
klm_conv['created_at']= pd.to_datetime(klm_conv['created_at'])
BA_conv['created_at']= pd.to_datetime(BA_conv['created_at'])
klm_conv = klm_conv.sort_values(['conversationnr', 'created_at'])
BA_conv = BA_conv.sort_values(['conversationnr', 'created_at'])
BA_conv = BA_conv.reset_index()
klm_conv = klm_conv.reset_index()
klm_conv['user.id_str'] = klm_conv['user.id_str'].astype(str)
BA_conv['user.id_str'] = BA_conv['user.id_str'].astype(str)

C:\Users\20173845\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#returns a orderedDict with the amount of conversations per length
def convlength(dataframe):
    dct= {}
    for i in range(len(dataframe)):
        if dataframe['conversationnr'][i] in dct:
            dct[(dataframe['conversationnr'][i])] += 1
        else:
            dct[(dataframe['conversationnr'][i])] = 1
    for i in list(dct.keys()):
        if dct[i] > 10 or dct[i] == 1:
            del dct[i]
    length= {}
    for i in range(len(dct)):
        if list(dct.values())[i] in length:
            length[list(dct.values())[i]] += 1
        else:
            length[list(dct.values())[i]] = 1
    
    sorted_x = sorted(length.items(), key=lambda kv: kv[0])
    sorted_dict = collections.OrderedDict(sorted_x)
    return sorted_dict

#visualisation of the dictionary made above
plt.bar(range(len(sorted_dict_klm)), list(sorted_dict_klm.values()))
plt.xticks(range(len(sorted_dict_klm)), list(sorted_dict_klm.keys()))
plt.xlabel('Conversation length')
plt.ylabel('Amount of conversations')
plt.title('Length of conversations klm', fontsize=16, weight='bold')
#plt.savefig('klm_conversations.png');

In [31]:
#just run it ^^
datetime_save = datetime.timedelta()
def function(entry):
    global datetime_save
    diff = entry - datetime_save
    datetime_save = entry
    return diff



In [36]:
#returns the dataframe with a new column reply_time
def replies(dataframe):
    datetime_save = datetime.timedelta()
    dataframe['reply_time'] = datetime.timedelta()
    dataframe['reply_time'] = np.array([function(entry) for entry in dataframe['created_at']])
    return dataframe

In [32]:
#give a dataframe and the user id of the airline
#it returns a dataframe with only replies from that airline, with the reply times and a column minutes
def reply_mean(dataframe, ID):
    dataframe['reply_time'] = datetime.timedelta()
    dataframe['reply_time'] = np.array([function(entry) for entry in dataframe['created_at']])
    airline_replies= dataframe[dataframe['user.id']== ID].set_index('index').reset_index()
    airline_reply = []
    for i in range(0,len(airline_replies)):
        if ((airline_replies['reply_time'][i] > e) and (airline_replies['reply_time'][i] <d)):
            airline_reply.append(airline_replies.loc[i])
    airline_reply = pd.DataFrame(airline_reply)
    
    airline_reply['minutes'] = airline_reply['reply_time'] / np.timedelta64(1, 'm')
    airline_reply_small = airline_reply.copy()
    airline_reply_small = airline_reply_small.reset_index()
    for i in range(len(airline_reply_small)):
        if airline_reply_small['minutes'][i]<0:
            airline_reply_small['minutes'][i]=airline_reply_small['minutes'][i]*-1
    max_klm=np.percentile(airline_reply['minutes'],90)
    min_klm=np.percentile(airline_reply['minutes'],10)
    airline_reply_small = airline_reply_small[airline_reply_small['minutes']<max_klm]
    airline_reply_small = airline_reply_small[airline_reply_small['minutes']>min_klm]
    airline_reply_small = airline_reply_small.drop('level_0',axis=1).reset_index()
    return airline_reply_small

datetime.timedelta(0)

In [34]:
#with this you can make new columns for which day the tweet was send, weekday can be changed to minutes, hours, days
dataframe['weekday']=dataframe['created_at'].dt.weekday

,index,Unnamed: 0,created_at,entities,favorite_count,filter_level,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,...,in_reply_to_user_id_str,lang,quoted_status,quoted_status_id_str,retweet_count,text,user.id,user.id_str,conversationnr,reply_time
0,0,4847915,2017-02-22 20:28:04,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.345000e+17,8.345000e+17,KLM,NaN,...,5.637714e+07,en,NaN,NaN,0.0,@KLM I received a storm warning email re cance...,9.130000e+02,913.0,1.0,-27 days +10:44:17
1,1,4889485,2017-02-23 05:19:05,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.346337e+17,8.346337e+17,alba,8.345000e+17,...,9.130000e+02,en,NaN,NaN,0.0,"@alba Hello Eric, we are sorry for the late re...",5.637714e+07,56377143.0,1.0,0 days 08:51:01
2,3,1849214,2016-08-31 13:27:55,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,7.709764e+17,7.709764e+17,cvodb,7.709639e+17,...,9.650000e+02,en,NaN,NaN,0.0,"@cvodb Out with the old, in with the new!",5.637714e+07,56377143.0,2.0,-176 days +08:08:50
3,4,2697745,2016-10-06 12:22:23,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,7.840059e+17,7.840059e+17,KLM,NaN,...,5.637714e+07,nl,NaN,NaN,0.0,@KLM ik kreeg mail dat ik gratis een standaard...,9.650000e+02,965.0,2.0,35 days 22:54:28
4,5,2703991,2016-10-06 12:32:05,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,7.840083e+17,7.840083e+17,cvodb,7.840059e+17,...,9.650000e+02,nl,NaN,NaN,0.0,"@cvodb Hoi Caroline, zou je ons je boekingscod...",5.637714e+07,56377143.0,2.0,0 days 00:09:42
5,2,2707643,2016-10-06 13:47:14,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,7.840272e+17,7.840272e+17,KLM,7.840083e+17,...,5.637714e+07,nl,NaN,NaN,0.0,"@KLM Dat kan, het is 43OTLK",9.650000e+02,965.0,2.0,0 days 01:15:09
6,7,5234013,2017-03-15 13:14:22,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.420010e+17,8.420010e+17,KLM,NaN,...,5.637714e+07,nl,NaN,NaN,0.0,@KLM Hoi! Ik heb in januari een naamswijziging...,1.189000e+03,1189.0,3.0,159 days 23:27:08
7,6,5234487,2017-03-15 13:34:05,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.420060e+17,8.420060e+17,charrlesmusic,8.420010e+17,...,1.189000e+03,nl,NaN,NaN,0.0,@charrlesmusic Stuur ons een DM met je boeking...,5.637714e+07,56377143.0,3.0,0 days 00:19:43
8,10,4850261,2017-02-24 09:34:24,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.350603e+17,8.350603e+17,KLM,NaN,...,5.637714e+07,en,NaN,NaN,0.0,@KLM due to the security madness at schiphol m...,2.631000e+03,2631.0,4.0,-20 days +20:00:19
9,8,4885255,2017-02-24 17:26:10,"{ ""hashtags"" : [ ], ""user_mentions"" : [ { ""sc...",0.0,low,8.351790e+17,8.351790e+17,thedaniel,8.350603e+17,...,2.631000e+03,en,NaN,NaN,0.0,"@thedaniel Hello, Daniel! Kindly send us your ...",5.637714e+07,56377143.0,4.0,0 days 07:51:46
